#### Dataset = kaggle competitions download -c digit-recognizer

## import dataset

In [102]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras.layers as tfl


In [103]:
train_df = pd.read_csv("../Datasets/digit-recognizer/train.csv")
test_df = pd.read_csv("../Datasets/digit-recognizer/test.csv")

In [104]:
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## preprocessing

In [105]:
# x_train = train_df.drop('label', axis=1).values / 255.0
# y_train = train_df['label'].values


In [156]:
# x_test = test_df.values / 255.0


In [157]:
# x_train.shape

In [158]:
x_all = train_df.drop('label', axis=1).values.astype('float32') / 255.0   
y_all = train_df['label'].values                                          
x_test = test_df.values.astype('float32') / 255.0 

In [159]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

x_train_flat, x_dev_flat, y_train, y_dev = train_test_split(
    x_all, y_all, test_size=4000, random_state=42, shuffle=True
)


x_train = x_train_flat.reshape(-1, 28, 28, 1)
x_dev   = x_dev_flat.reshape  (-1, 28, 28, 1)
x_test  = x_test.reshape     (-1, 28, 28, 1)


y_train_enc = to_categorical(y_train, num_classes=10)
y_dev_enc   = to_categorical(y_dev,   num_classes=10)

In [160]:

# # Create iterators
# images_iter = iter(x_train_reduced)
# labels_iter = iter(y_train_reduced)

# # Plot 25 samples
# plt.figure(figsize=(10, 10))
# for i in range(25):
#     ax = plt.subplot(5, 5, i + 1)
#     plt.imshow(next(images_iter).reshape(28, 28), cmap='gray')  # Reshape for MNIST (28x28)
#     plt.title(int(next(labels_iter)))  # Convert label to integer
#     plt.axis("off")
# plt.show()

In [161]:
from tensorflow.keras import regularizers

# model = tf.keras.models.Sequential([
#     tf.keras.Input(shape=(x_train_reduced.shape[1],)),
#     tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
#     tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
#     tf.keras.layers.Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
#     tf.keras.layers.Dense(y_train_encoded.shape[1], activation='softmax') ,

# ])

model = tf.keras.Sequential([
    tfl.ZeroPadding2D(padding=3, input_shape=(28,28,1)),
    tfl.Conv2D(32, (5,5), strides=1, padding='same'),
    # tfl.BatchNormalization(),
    tfl.ReLU(),
    tfl.MaxPooling2D(pool_size=2, strides=2),

    tfl.Conv2D(16, (5,5), strides=1),
    # tfl.BatchNormalization(),
    tfl.ReLU(),
    tfl.MaxPooling2D(pool_size=2, strides=1),

    tfl.Conv2D(8, (3,3), strides=1),
    tfl.BatchNormalization(),
    tfl.ReLU(),
    tfl.Flatten(),

    tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
    tf.keras.layers.Dense(10, activation='softmax'),
])

# tfl.ZeroPadding2D(padding=3, input_shape=(64,64,3)),
# tfl.Conv2D(filters=32, strides=1, kernel_size=(7,7)),
# tfl.BatchNormalization(axis=3),
# tfl.ReLU(),
# tfl.MaxPool2D(),
# tfl.Flatten(),
# tfl.Dense(units=1, activation='sigmoid')

/home/mra/Desktop/machine-learning-practice/venv/lib/python3.12/site-packages/keras/src/layers/reshaping/zero_padding2d.py:72: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [162]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=100000,
    decay_rate=0.9
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)


In [163]:
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)



In [164]:
print(x_train.shape, y_train_enc.shape)
assert x_train.shape[0] == y_train_enc.shape[0] > 0, "Empty or mismatched data!"


(38000, 28, 28, 1) (38000, 10)


In [165]:
dev_loss, dev_acc = model.evaluate(x_dev, y_dev_enc, verbose=0)
print(f"Before training → dev accuracy = {dev_acc:.4f}")

Before training → dev accuracy = 0.1470


In [167]:
model.fit(
    x_train, y_train_enc,
    epochs=50,
    batch_size=128,
    validation_data=(x_dev, y_dev_enc)
)

Epoch 1/50
297/297 ━━━━━━━━━━━━━━━━━━━━ 15s 52ms/step - accuracy: 0.9660 - loss: 0.1278 - val_accuracy: 0.9740 - val_loss: 0.1008
Epoch 2/50
297/297 ━━━━━━━━━━━━━━━━━━━━ 15s 51ms/step - accuracy: 0.9822 - loss: 0.0715 - val_accuracy: 0.9720 - val_loss: 0.1091
Epoch 3/50
297/297 ━━━━━━━━━━━━━━━━━━━━ 15s 52ms/step - accuracy: 0.9868 - loss: 0.0569 - val_accuracy: 0.9827 - val_loss: 0.0687
Epoch 4/50
297/297 ━━━━━━━━━━━━━━━━━━━━ 15s 51ms/step - accuracy: 0.9885 - loss: 0.0549 - val_accuracy: 0.9682 - val_loss: 0.1149
Epoch 5/50
297/297 ━━━━━━━━━━━━━━━━━━━━ 15s 52ms/step - accuracy: 0.9901 - loss: 0.0476 - val_accuracy: 0.9758 - val_loss: 0.1046
Epoch 6/50
297/297 ━━━━━━━━━━━━━━━━━━━━ 15s 52ms/step - accuracy: 0.9890 - loss: 0.0495 - val_accuracy: 0.9847 - val_loss: 0.0684
Epoch 7/50
297/297 ━━━━━━━━━━━━━━━━━━━━ 15s 52ms/step - accuracy: 0.9904 - loss: 0.0458 - val_accuracy: 0.9815 - val_loss: 0.0812
Epoch 8/50
297/297 ━━━━━━━━━━━━━━━━━━━━ 16s 52ms/step - accuracy: 0.9897 - loss: 0.0491 - 

In [168]:
dev_loss, dev_acc = model.evaluate(x_dev, y_dev_enc, verbose=0)
print(f"After training → dev accuracy = {dev_acc:.4f}")

After training → dev accuracy = 0.9893


In [ ]:
import numpy as np
y_test_probs = model.predict(x_test)
y_test_pred  = np.argmax(y_test_probs, axis=1)

619/875 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step

In [120]:
submission_df = pd.DataFrame({
    'ImageId': range(1, len(y_test_pred) + 1),
    'Label': y_test_pred
})

submission_df.to_csv('../Datasets/digit-recognizer/submission.csv', index=False)
